In [1]:
from src.data.datagen import DataFetcher
import os

In [2]:
datagen = DataFetcher()

In [3]:
schemas = ["bbo-1m", "imbalance", "ohlcv-1m"]

merges = datagen.join_cache(schemas)

Working on: xnas-itch-20180501-20250314.ohlcv-1m.parquet
Joining AMD_ohlcv-1m.parquet
Joining NVDA_ohlcv-1m.parquet
Joining INTC_ohlcv-1m.parquet


In [ ]:
# import databento as db
# import os



# path = 'data/temp/xnas-itch-20180501-20250314.ohlcv-1m.dbn.zst'

# with open(path, "rb") as saved:
#     stored_data = db.DBNStore.from_bytes(saved).to_df()

# stored_data.to_parquet("data/raw/xnas-itch-20180501-20250314.ohlcv-1m.parquet")